In [5]:
import pandas as pd
from tqdm.contrib.concurrent import process_map
import pathlib

BASE_URL = 'https://dom.mingkh.ru'
BASE_FILEPATH = 'base_home_data.csv'
FULL_FILEPATH = 'complete_home_info.csv'


def parse_home_page(home_url: str):
    full_url = BASE_URL + home_url
    _id = home_url.split('/')[-1]
    df_list = pd.read_html(full_url)
    home_data = pd.concat(df_list)\
        .iloc[:, :3]\
        .dropna(thresh=1)\
        .set_index(0)\
        .fillna(method='ffill', axis=1)\
        .iloc[:, 1]\
        .rename(_id)\
        .rename_axis('id')
    return home_data.to_frame().T



In [7]:
tmp_ = pd.read_pickle('/home/charubaiel/workflow/projects/MLOpsini_project/pipelines/.DH/storage/cian_dataframe')

In [22]:
tmp_[['url','publish_delta']].drop_duplicates()

,url,publish_delta
0,https://www.cian.ru/sale/flat/286864465/,1 минуту назад
1,https://www.cian.ru/sale/flat/286864404/,25 минут назад
2,https://www.cian.ru/sale/flat/286864388/,25 минут назад
3,https://www.cian.ru/sale/flat/286864229/,30 минут назад
4,https://www.cian.ru/sale/flat/286864094/,45 минут назад
...,...,...
81,https://www.cian.ru/sale/flat/286856140/,45 минут назад
82,https://www.cian.ru/sale/flat/286856128/,12 часов назад
83,https://www.cian.ru/sale/flat/286856006/,12 часов назад
602,https://www.cian.ru/sale/flat/286863711/,час назад


In [12]:
def title_features(cian_dataframe:pd.DataFrame) -> pd.DataFrame:

    result = {}
    title_series = cian_dataframe['title'].str.lower().to_frame()
    title_series[['rooms','m2','floor']] = title_series.str.replace(',(?=\d)','.').str.split(',',expand=True)

    title_series[['floor','max_floor']] = title_series['floor'].str.extract('(\d+/\d+).*эт').iloc[:,0].str.split('/',expand=True).astype(float)
    title_series['m2'] = title_series['m2'].str.extract('(\d+).*м²').astype(float)
    title_series['is_max_floor'] = title_series['floor'] == title_series['max_floor']


    return title_series

In [13]:
title_features(tmp_)

0 1 2


/tmp/ipykernel_292/1616650272.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  rooms,m2,floor = title_series.str.replace(',(?=\d)','.').str.split(',',expand=True)


AttributeError: 'int' object has no attribute 'str'

In [3]:
with duckdb.connect('../data/protodb.db') as conn:
    df = conn.query('select * from intel.cian').df().drop_duplicates(subset=['url'])